In [12]:
from importlib import reload
import flymovie as fm
import cnn_models
import cnn_models.siamese_cnn as cn
import cnn_models.evaluate_models as ev
from flymovie.simnuc import Sim
import matplotlib.pyplot as plt
import numpy as np
import os
from fpdf import FPDF
import random
import pickle
from pathlib import Path
import scipy.spatial
import scipy.ndimage as ndi
import tensorflow as tf
from importlib import reload
%load_ext autoreload
%autoreload 2

In [59]:
from cnn_models.siamese_cnn import preprocess_image


In [98]:
from cnn_models.siamese_cnn import preprocess_image
from tensorflow.keras import layers
from tensorflow.keras import Model
#---------------------------------------------------------------------------
def get_target_shape(dir_):
    """Retrieve the image target size."""
    left = dir_ / 'left'
    imfile = os.listdir(left)[-1]
    impath = os.path.join(left, imfile)
    with open(impath, 'rb') as file:
        im = pickle.load(file)

    shape = im.shape
    return shape

def make_im_param_inputs(folder, batch_size=32):
    """
    """
    def get_files_params(subfolder):
        files = os.listdir(subfolder)
        files = [x if x[0] != '.' else None for x in files]
        n_params = len(files[0].split('_')) - 2
        params = np.zeros((0, n_params))
        for f in files:
            p = f.split('_')[1:-1]
            p = [float(x) for x in p]
            p = np.expand_dims(np.array(p), axis=0)
            params = np.vstack((params, p))
        files = [os.path.join(subfolder, x) for x in files]
        return files, list(params)

    def preprocess(file, paramset):
        [im,] = tf.py_function(preprocess_image,[file,],[tf.float32,])
        return im, paramset

    # Set up input directories.
    cache_dir=folder
    left_path = cache_dir / "left"
    right_path = cache_dir / "right"
    
    files_l, params_l = get_files_params(left_path)
    files_r, params_r = get_files_params(right_path)
    files = files_l + files_r
    params = params_l + params_r

    dataset = tf.data.Dataset.from_tensor_slices((files, params))
    dataset = dataset.shuffle(buffer_size=len(files))

    dataset = dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(32)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset
    # Create datasets from these sorted files. These are in order and match in pairs.
    
ds = make_im_param_inputs(Path('/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/testsims_uPoNivMJ_10'))

In [97]:
target_shape = get_target_shape(cache_dir)
base_cnn = cn.make_base_cnn_3d(target_shape, 'base_cnn', nlayers=8)

flatten = layers.Flatten()(base_cnn.output)
dense1 = layers.Dense(512, activation="relu")(flatten)
dense1 = layers.BatchNormalization()(dense1)
dense2 = layers.Dense(256, activation="relu")(dense1)
dense2 = layers.BatchNormalization()(dense2)
output = layers.Dense(9)(dense2)

In [102]:
model = Model(base_cnn.input, output, name="Model")
model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=["acc"],
)


In [104]:
model.fit(
    ds,
    epochs=5,
    shuffle=True,
    verbose=2
)

Epoch 1/5
1/1 - 12s - loss: 3737.2253 - acc: 0.0500
Epoch 2/5
1/1 - 9s - loss: 11794.1465 - acc: 0.2000
Epoch 3/5
1/1 - 9s - loss: 6266.0391 - acc: 0.3500
Epoch 4/5
1/1 - 9s - loss: 6394.3345 - acc: 0.3500
Epoch 5/5
1/1 - 9s - loss: 5369.0830 - acc: 0.2500


In [10]:
import numpy as np
import scipy.spatial

num_to_sample = 100_000
files = np.arange(100_000)
a1 = np.random.random(9)
a2 = np.random.random(9)
rs = np.random.RandomState()
for _ in range(num_to_sample):
    rs = np.random.RandomState()
    rs.choice(files)
    rs.choice(files)
    (a1 - a2) / a1
    (a1 - a2) / a1
    dist = scipy.spatial.distance.euclidean(a1, a2)


In [130]:
train, val = cn.make_triplet_inputs('/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/testsims_uPoNivMJ/filetriplets_95.0_100.0_5_qvs.csv.gz', batch_size=50, epoch_size=50, rotate=False)
ev.visualize_batch(train)

interactive(children=(Dropdown(description='Color', index=8, options=('viridis', 'inferno', 'magma', 'Gators',…

In [129]:
train, val = cn.make_triplet_inputs('/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/testsims_uPoNivMJ/filetriplets_0.0_4.0_5_cpm.csv.gz', batch_size=50, epoch_size=50, rotate=False)
ev.visualize_batch(train)

interactive(children=(Dropdown(description='Color', index=8, options=('viridis', 'inferno', 'magma', 'Gators',…

In [90]:
a = np.random.random((100_000, 9))
#np.sqrt(np.sum((a - a[0,:]) ** 2, axis=1))
b = np.sqrt(np.sum(((a - a[0,:]) ** 2), axis=1))
np.where(b > 0)[0]

array([    1,     2,     3, ..., 99997, 99998, 99999])

In [100]:
np.random.choice(np.arange(100))

36

In [87]:
a = np.random.random((100_000, 9))
for i in range(a.shape[0]):
    if i % 1_000 == 0:
        print(i)
    b = np.sqrt(np.sum(((a - a[0,:]) ** 2), axis=1))


0
1000
2000


KeyboardInterrupt: 

np.random.randint(100_000)

In [101]:
import random
import string

nfiles = 10
filenames = []
for _ in range(nfiles):
    filename = ''.join([random.choice(string.ascii_letters) for i in range(3)])
    for i in range(8):
        n = random.choice([1,2,3,4,5,6,7,8,9])
        filename = filename + '_' + str(n)
    filename = filename + '_rep0.pkl'
    filenames.append(filename)

In [127]:
#cn.match_file_triplets(filenames, filenames)

In [29]:
a = fm.load_pickle("/Users/michaelstadler/Bioinformatics/Projects/rpb1/results/testsims_uPoNivMJ_10/filetriplets_0.0_100.0_FMV.pkl")